<p style ="font-size:30px; font-weight:bold; color:green"> 데이터청년캠퍼스 플랜웍스팀 파이썬 코드</p>

<hr>

# 데이터 수집

## Google map Crawling code

In [ ]:
# selenium version setting
# !pip install --upgrade selenium==3.141.0

# import package
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By  # 3개는 selenium이 동작하면서 웹 화면이 로딩될때 까지 기다림을 지원
from selenium.webdriver.support.ui import WebDriverWait # 웹 드라이버 미리 설치
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# run in chromedriver
driver = webdriver.Chrome('C:/Users/chromedriver.exe')
# enter review url
url = 'https://www.google.com/maps/place/%ED%9B%84%EC%A7%84%ED%95%AD/@38.132737,128.6213,17z/data=!4m11!1m2!2m1!1z7ZuE7KeE7ZWt!3m7!1s0x5fd8aff6e709b405:0x7b04ed680c3c22f4!8m2!3d38.132886!4d128.6226878!9m1!1b1!15sCgntm4Tsp4Ttla2SAQZtYXJpbmE?hl=en'
driver.get(url)
driver.maximize_window()
time.sleep(5)

# sort to most relevant review 
wait = WebDriverWait(driver, 10)
menu_bt = wait.until(EC.element_to_be_clickable(
                       (By.XPATH, '//button[@data-value=\'Sort\']'))
                   )  
menu_bt.click()

recent_rating_bt = driver.find_elements_by_xpath(
                                     '//li[@role=\'menuitemradio\']')[0]
recent_rating_bt.click()
time.sleep(5)


# scroll code
# total n review (10 reviews per 1 page)
from tqdm import tqdm

page_num = 50
per = 10
pbar = tqdm(total=page_num*per)
with tqdm(total=page_num*per, position=0, leave=True) as pbar:
    for i in range(page_num):
        buffer = driver.find_elements_by_class_name('qjESne') # buffer location
        action = ActionChains(driver)

        n =len(buffer)
        action.move_to_element(buffer[n-1]).perform() # move to last buffer
        time.sleep(1)
        pbar.update(per)
    pbar.close()

# review item list    
box_list = []
name_list = []
stars_list = []
review_list = []
date_list = []

boxes = driver.find_elements_by_class_name('jftiEf.fontBodyMedium') # review box class

# more butten click
for box in boxes:
    try:
        more_btn = box.find_element_by_class_name('w8nwRe.kyuRq')
        more_btn.click()
        time.sleep(1)
    except:
        continue
        
# review collection
for box in boxes:
    # each class location
    IDs = box.find_elements_by_class_name('d4r55')
    Dates = box.find_elements_by_class_name('rsqaWe')
    Stars = box.find_elements_by_class_name('kvMYJc')
    reviews = box.find_elements_by_css_selector('span.wiI7pd')

    # append to list after each review collection
    for ID, Date, Star, review in zip(IDs, Dates, Stars, reviews):
        name_list.append(ID.text)
        date_list.append(Date.text)
        stars_list.append(Star.get_attribute('aria-label'))
        review_list.append(review.text)
        
# review list to dataframe
review = pd.DataFrame(
    {'name': name_list,
     'rating': stars_list,
     'review': review_list,
     'date': date_list
    })

# check number of review
print(len(review))
print(review.tail())


# We use only foreign tourist. So deletes data with Korean in names and reviews

eng_review = review.copy()

# remove korean name
eng_review['name2'] = eng_review['name'].str.findall(r'[ㄱ-ㅎㅏ-ㅣ가-힣]')
eng_review = eng_review[eng_review['name2'].str.len() == 0]

# remove korean review
eng_review['review2'] = eng_review['review'].str.findall(r'[ㄱ-ㅎㅏ-ㅣ가-힣]')
eng_review = eng_review[eng_review['review2'].str.len() == 0]

# remove name2 & review2 columns
eng_review.drop(['name2','review2'],axis=1, inplace=True)

# remove "(Translated by Google)" sentence
eng_review['review'] = eng_review['review'].str.replace('(Translated by Google) ','',regex=False)

# remove "Original" word
eng_review['review'] = eng_review['review'].str.replace('\(Original\)','',regex=False)

# remove not english sentence & special characters
eng_review['review'] = eng_review['review'].str.replace(r'[^A-Za-z0-9]',' ', regex=True)

# delete side blank and "\n"
eng_review['review'] = eng_review['review'].str.strip()

# replace with lowercase
eng_review['review'] = eng_review['review'].str.lower()

# remove "orginal" word
eng_review['review'] = eng_review['review'].str.replace('original','')

# remove unreviewed data
eng_review = eng_review[eng_review['review'].str.len()!=0]

print("number of review:", len(eng_review))
print(eng_review.tail(10))

# Finally, We add detailed information
eng_review['attraction'] = 'attraction'
eng_review['address'] = 'address'
eng_review['latitude'] = 36.7823029
eng_review['longitude'] = 126.4591244
#eng_review.to_csv("양양/동호해수욕장.csv", encoding="utf-8", index = None) # 맥북 기준
eng_review.to_csv("양양/동호해수욕장.csv", encoding="utf-8", index = None) # Window 기준

## Trip Advisor Crawling code

In [ ]:
# selenium version setting
# !pip install --upgrade selenium==3.141.0

# import package

import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By  # 3개는 selenium이 동작하면서 웹 화면이 로딩될때 까지 기다림을 지원
from selenium.webdriver.support.ui import WebDriverWait # 웹 드라이버 미리 설치
from selenium.webdriver.support import expected_conditions as EC # support안에 있는 expected_conditions를 EC로 선언

In [ ]:
# run in chromedriver
driver = webdriver.Chrome('C:/Users/chromedriver.exe')
# enter review url
url = "https://www.tripadvisor.com/Attraction_Review-g612375-d3805539-Reviews-Naksansa_Temple-Yangyang_gun_Gangwon_do.html"
driver.get(url)
driver.maximize_window()
time.sleep(5)

# review item list 
box_list = []
name_list = []
stars_list = []
review_list = []
date_list = []

num_page = 5
i=0
while (i<num_page):
    # review box class
    boxes = driver.find_elements_by_css_selector('div._c')

    for box in boxes:
        try:
            # more button click
            more_btn = box.find_element_by_class_name('biGQs._P.XWJSj.Wb')
            more_btn.click()
            time.sleep(5)
        except:
            continue
            
    # review collection
    for box in boxes:
        # each class location
        IDs = box.find_elements_by_css_selector('span.biGQs._P.fiohW.fOtGX')
        Dates = box.find_elements_by_class_name('biGQs._P.pZUbB.ncFvv.osNWb')
        Stars = box.find_elements_by_css_selector('svg.UctUV.d.H0')      
        reviews = box.find_elements_by_class_name('biGQs._P.pZUbB.KxBGd')

        # append to list after each review collection
        for ID,Date,Star,review in zip(IDs, Dates, Stars, reviews):
            name_list.append(ID.text)
            date_list.append(Date.text)
            stars_list.append(Star.get_attribute('aria-label'))
            review_list.append(review.text)

    time.sleep(10)
    try:
        driver.find_element_by_class_name(a.BrOJk.u.j.z._F.bYExr.tIqAi.unMkR).click()
        time.sleep(5)
    except:
        break
    i += 1
    
review_data = pd.DataFrame(
    {'name': name_list,
     'rating': stars_list,
     'review': review_list,
     'date': date_list})

# remove unnecessary words
eng_review = review_data.copy()

# remove not english sentence & special characters
eng_review['review'] = eng_review['review'].str.replace(r'[^A-Za-z0-9]',' ', regex=True)

# delete side blank and "\n"
eng_review['review'] = eng_review['review'].str.strip()

# replace with lowercase
eng_review['review'] = eng_review['review'].str.lower()

#date에서 Written 없애기
eng_review['date'] = eng_review['date'].str.replace('Written ','')

print("number of review:", len(eng_review))

# Finally, We add detailed information
eng_review['attraction'] = 'attraction'
eng_review['address'] = 'address'
eng_review['latitude'] = 38.1253749
eng_review['longitude'] = 128.6307526
#eng_review.to_csv("양양/의상대_trip.csv", encoding="utf-8", index = None) # 맥북 기준
eng_review.to_csv("양양/의상대_trip.csv", encoding="cp949", index = None) # Window 기준

<span style= "color:blue"> 이후 엑셀로 추가작업하였습니다.</span>

# 데이터 전처리

* [Colab][1]으로 전처리를 수행해 해당 링크 안에서 실행하는 것을 권장합니다.
[1]: https://colab.research.google.com/drive/1e8DpXuwMO1WTEBxRKqEhwFvhubNyk05y?usp=sharing

## 단어 토큰화 전처리

In [2]:
import pandas as pd
import numpy 
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
import random

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YOONSU\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\YOONSU\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\YOONSU\AppData\Roaming\nltk_data...
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\YOONSU\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping help\tagsets.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\YOONSU\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [ ]:
data = pd.read_csv('/content/date17-22_data.csv')
data = pd.DataFrame(data)

#단어 토큰화 전처리 코드
text_data = data['review']
text_data = text_data.str.replace(r'[^ a-zA-Z]',' ', regex=True) #숫자 제거
#[a-zA-Z] #[^A-Za-z0-9] 

def data_text_cleaning(data):
    tokens = data.split() #토큰화
    stops = set(stopwords.words('english')) #불용어 제거
    no_stops = [word for word in tokens if not word in stops] 
    return ' '.join(no_stops) #공백으로 구분된 문자열로 결합하여 결과 반환


text_list = list()
tokens_list = list()
for i in range(len(text_data)):
    text_list.append(data_text_cleaning(text_data[i]))
    tokens_list.append(text_list[i].split())
#print(tokens_list) #list 형태
data['tokens'] = tokens_list

#두글자 제거
noun_list = data['tokens']
for i,v in enumerate(noun_list):
    for j,k in enumerate(v):
        if len(k)<3:
            del noun_list[i][j]

#품사 부착
tokens = data['tokens']
tokens_pos_list = []
for l in tokens:
    pt = pos_tag(l)
    tokens_pos_list.append(pt)

#품사부착 + 원형복원
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('V'):
        return 'v'
    elif pos_tag.startswith('N'):
        return 'n'
    elif pos_tag.startswith('J'):
        return 'a'
    elif pos_tag.startswith('R'):
        return 'r'
    else:
        return None

#pos_tag가 반환한 품사를 원형복원용 품사로 변환
tokens_pos_list2 = []
for token_list in tokens_pos_list:
    temp_list = []
    for token, pos_tag in token_list:
        tag = get_wordnet_pos(pos_tag)
        if tag!=None: #변환된 품사가 None이 아닌 것들만 list에 담는다.
            temp_list.append((token, get_wordnet_pos(pos_tag)))
    tokens_pos_list2.append(temp_list)
    
#원형복원(lemm.lemmatize)
lemm = WordNetLemmatizer()
token_final = []
for token_pos in tokens_pos_list2:
    tmp_list = []
    for token, tag in token_pos:
        tmp_list.append(lemm.lemmatize(token, pos=tag))
    token_final.append(tmp_list)

data = data.assign(new_tokens = token_final)

#new_tokens 빈 리스트를 가진 행 지우기
locate = []
for i in range(len(data['review'])):
    if data['new_tokens'][i] == []:
        locate.append(i)
print(locate)

data = data.drop(locate, axis=0)
data.shape #(4996, 11)

#index reset
data.reset_index(drop=True, inplace=True)

#토큰 하나인 리뷰 index 제거
locate = []
for i in range(len(data['review'])):
    if len(data['new_tokens'][i]) == 1:
        locate.append(i)

data = data.drop(locate, axis=0)
data.shape #(4485, 11)

#index reset
data.reset_index(drop=True, inplace=True)

## 워드클라우드

In [ ]:
tokens_list = data['new_tokens']
str = numpy.concatenate(tokens_list).tolist() #전체 리뷰 문장 토큰화
c = Counter(str)
c2 = c.most_common(100)
print(c2)
c3 = dict(c2); c3
print(len(dict(Counter(str)))) #단어 table

stopwords = set(STOPWORDS)
wordcloud = WordCloud(width = 800, height = 800,
                      stopwords = stopwords,
                      background_color="white",
                      min_font_size = 10).generate_from_frequencies(c3)

plt.figure(figsize = (10,15), facecolor = None)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
# 불필요한 단어 제거

rm = ['lot', 'bit', 'day', 'area', 'place', 'one', 'view','visit', 'sokcho', 'gangneung', 'spot', 'trip', 'way', 'people', 
      'thing', 'get', 'part', 'st', 'km', 'pm', 'chi', 'krw', 'thru', 'throll', 'also','good','great']
for i,v in enumerate(data['new_tokens']):
    for j,k in enumerate(v):
        if k in rm:
            del data['new_tokens'][i][j]

In [ ]:
#다시 워드클라우드

tokens_list = data['new_tokens']
str = numpy.concatenate(tokens_list).tolist() #전체 리뷰 문장 토큰화
c = Counter(str)
c2 = c.most_common(50)
print(c2)
c3 = dict(c2); c3
len(dict(Counter(str))) #중복되지 않는 단어의 개수

stopwords = set(STOPWORDS)
wordcloud = WordCloud(width = 800, height = 800,
                      stopwords = stopwords,
                      background_color="white",
                      min_font_size = 10).generate_from_frequencies(c3)

plt.figure(figsize = (10,15), facecolor = None)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

# 데이터 모델링

## LDA 토픽모델링

* [Colab][1]으로 LDA 토픽모델링을 수행해 해당 링크 안에서 실행하는 것을 권장합니다.
[1]: https://colab.research.google.com/drive/1e8DpXuwMO1WTEBxRKqEhwFvhubNyk05y?usp=sharing

In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric
import logging

!pip install pyLDAvis
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import gensim
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [ ]:
processed_data = data['new_tokens']

dictionary = corpora.Dictionary.load("/content/dic.csv")
dictionary.filter_extremes(no_below=5, no_above=0.5) 
corpus = [dictionary.doc2bow(text) for text in processed_data]

In [ ]:
#preplexity 값: 확률 모델이 결과를 얼마나 정확하게 예측하는지/ 낮을수록 정확하게 예측
'''chunksize = 1000 
passes = 20
iterations = 400
eval_every = None

preplexity_values=[]
for i in range(2,15):
  ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=dictionary, random_state=40,
                                           chunksize=chunksize, passes = passes, iterations=iterations, eval_every=eval_every)
  preplexity_values.append(ldamodel.log_perplexity(corpus))

x=range(2,15)
plt.plot(x, preplexity_values)
plt.xlabel("number of topics")
plt.ylabel("perplexity score")
plt.show() #6~8에서 완만해짐'''

In [ ]:
#coherence 값: 토픽이 얼마나 의미론적으로 일관성 있는지/ 높을수록 의미론적 일관성 높음
'''coherence_values=[]
for i in range(2,15):
  ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=dictionary, random_state=40,
                                           chunksize=chunksize, passes = passes, iterations=iterations, eval_every=eval_every)
  coherence_model_lda=CoherenceModel(model=ldamodel, texts=data['new_tokens'], dictionary=dictionary, topn=10)
  coherence_lda=coherence_model_lda.get_coherence()
  coherence_values.append(coherence_lda)

x=range(2,15)
plt.plot(x, coherence_values)
plt.xlabel("number of topics")
plt.ylabel("coherence score")
plt.show() #topic 6이 최적
'''

In [ ]:
chunksize = 1000 
passes = 20
iterations = 400
eval_every = None
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=6, id2word=dictionary, random_state=40,
                                           chunksize=chunksize, passes = passes, iterations=iterations, eval_every=eval_every)
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print("topic 별 단어:", topic)
    
x =[]
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    x.append(topic[1])
print("topic별 단어리스트:", x)

# LDA visualization
vis = gensimvis.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(vis, 'topic6_last.html')

In [ ]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

data['가장 비중이 높은 토픽'] = topictable['가장 비중이 높은 토픽']
data.rename(columns={"가장 비중이 높은 토픽": "topic"}, inplace=True)
data.to_csv('topic6_data.csv', index = False, encoding = 'utf-8')

## 토픽별 상위 10개 단어 시각화

In [ ]:
import seaborn as sns
fiz=plt.figure(figsize=(30,15))
plt.rc('font', family='NanumBarunGothic')

for i in range(6):
    df=pd.DataFrame(ldamodel.show_topic(i), columns=['term','prob']).set_index('term')
    
    plt.subplot(2,3,i+1)
    plt.rc('xtick', labelsize=15)
    plt.rc('ytick', labelsize=15)
    c_list = ['lightcoral','mediumseagreen','darkkhaki','darkturquoise','orchid','cornflowerblue']
    t_name = ['식도락 관광', '워터투어', '산행하기 좋은 곳', '낭만적인 장소', '한국의 문화체험관광', '에코 힐링']
    plt.title('topic {0} : {1}'.format(i+1, t_name[i]),fontweight="bold",size=18)
    sns.barplot(x='prob', y=df.index, data=df, color=c_list[i])
    plt.xlabel('probability',size=14)
    plt.ylabel('term',size=14)

plt.savefig('savefig_default.png')

## 명소별 토픽 선정

* 명소별 토픽 선정 후 시각화는 [태블로][1]로 작업해 해당 링크에서 확인할 수 있습니다.
[1]: https://public.tableau.com/app/profile/.74155837/viz/___16613166394110/1

In [ ]:
df = pd.read_csv("topic6_data.csv", encoding='cp949')
print(df.head())
print(df.topic.value_counts())

# 리뷰별 토픽 최빈값으로 명소별 토픽 추출
df2 = pd.DataFrame(df.groupby(['attraction','latitude','longitude','address']).topic.apply(lambda x: x.mode()))

df2.reset_index(inplace=True)
df2.drop('level_4', axis=1, inplace=True)

# 최빈값이 두개이상일때 마지막 값으로 표시
df2 = df2[df2['attraction'].duplicated(keep='last')==False]

df2.to_csv("topic6_unique.csv", encoding="cp949")

# 평가

## 입력한 단어가 속한 데이터보기

In [ ]:
import pandas as pd
import re

df = pd.read_csv("/root/topic6_data_final.csv", encoding="cp949")
print(df.head())

df.word = df.new_tokens.str.findall('expensive') # input the word you want to find
print(df[df.word.str.len() != 0].shape)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df_len = len(df[df.word.str.len() != 0])

df2 = df[df.word.str.len() != 0][['attraction','review','new_topic']]
df2[df2['new_topic']==1] # input the topic you want to find